Testing of ResNet+MLP where we take the ResNet output, concatenate it with a gravity vector (which the drone has via its IMU) and pass it through an MLP to get an estimate of the pose.

We try out a bunch of hidden layer configurations and also compare results between networks with pre-trained ResNet weights and networks where the ResNet weights are allowed to be trained on given the data.

In [1]:
# Some useful settings for interactive work
%load_ext autoreload
%autoreload 2

In [2]:
import functions.data as df
import functions.training as tf
from models import *

In [3]:
# Initialize NN variables
hidden_sizes = [
    [256, 128, 7],
    [128, 32, 7],
    ]

mlps = [VisionPoseMLP(hidden_size,False) for hidden_size in hidden_sizes]

Neps = 500
Ndata = None
ratio = 0.8
Nexp = 0

In [5]:
# Generate data loaders
train_loader, test_loader = df.get_data(ratio,Ndata)

(1, 3, 224, 224)
(1, 3, 224, 224)
(1, 3, 224, 224)


In [6]:
# Train the basic models
for idx,mlp in enumerate(mlps):
    mlp_name = "basic"+str(idx+1).zfill(3)
    print("=============================================================")
    print("Training: "+mlp_name)
    tf.train_model(mlp, train_loader, mlp_name,useNeRF=False, Neps=Neps)
    print("-------------------------------------------------------------")
    tf.test_model(mlp, test_loader,useNeRF=False, Nexp=Nexp)

# print(mlp.networks[0].model[0].conv1.weight[0,0,0,:])

Training: basic001
Epoch:  50 | Loss: 0.02507
Epoch: 100 | Loss: 0.02491
Epoch: 150 | Loss: 0.02902
Epoch: 200 | Loss: 0.02062
Epoch: 250 | Loss: 0.03973
Epoch: 300 | Loss: 0.02569
Epoch: 350 | Loss: 0.01474
Epoch: 400 | Loss: 0.01453
Epoch: 450 | Loss: 0.01099
Epoch: 500 | Loss: 0.00896
-------------------------------------------------------------
-------------------------------------------------------------
Test Loss: 0.0170
Examples:
Training: basic002
Epoch:  50 | Loss: 0.03154
Epoch: 100 | Loss: 0.02505
Epoch: 150 | Loss: 0.02022
Epoch: 200 | Loss: 0.02337
Epoch: 250 | Loss: 0.01700
Epoch: 300 | Loss: 0.02098
Epoch: 350 | Loss: 0.01540
Epoch: 400 | Loss: 0.02224
Epoch: 450 | Loss: 0.01754
Epoch: 500 | Loss: 0.01851
-------------------------------------------------------------
-------------------------------------------------------------
Test Loss: 0.0245
Examples:
